In [1]:
import pandas as pd

pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)

# model2

In [2]:
import numpy as np

train = pd.read_csv('./data/train.csv').drop('index',axis=1)

# family size 이상치
train = train.drop(1019)
# age 이상치
train = train.drop(train[train['age']>100].index.to_list())
train = train.drop(train[train['familysize']>100].index.to_list())

test_index = pd.read_csv('./data/test.csv')['index']

value = train['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))

temp_dict = {i : 0 for i in train['country'].value_counts().index.to_list()}

rank_dict = dict(zip(train['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
train['country'] = train['country'].fillna('nan')
train['country'] = train['country'].apply(lambda x : rank_dict[x])
train['Ex'] = (train['TIPI1']+train['TIPI6'])/2
train['Ag'] = (train['TIPI7']+train['TIPI2'])/2
train['Con'] = (train['TIPI3']+train['TIPI8'])/2
train['Es'] =(train['TIPI9']+train['TIPI4'])/2
train['Op'] =(train['TIPI5']+train['TIPI10'])/2

###
train['TIPI_normal'] = train['TIPI1'].apply(lambda x : 1 if x==4 else 0)
train['TIPI_normal'] += train['TIPI2'].apply(lambda x : 1 if x==2 else 0)
train['TIPI_normal'] += train['TIPI10'].apply(lambda x : 1 if (x==3 or x== 4) else 0)
train['Es_gender'] = train['Es']*train['gender']
train['Con_gender'] = train['Con']*train['gender']
train['Op_gender'] = train['Op']*train['gender']
# train['religion'] = train['religion'].apply(lambda x : 1 if (x==1 or x==2) else 2)
train['religion'] = np.log1p(train['religion'])

train['mach_score'] = train[train.columns[:20]].apply(lambda x : x.mean(),axis=1)
train['T'] = train['Q1'] + train['Q2'] + train['Q3'] + train['Q6'] + train['Q7'] + train['Q10'] + train['Q12'] + train['Q15'] + train['Q16']
train['V'] = train['Q4'] + train['Q5'] + train['Q8'] + train['Q11'] + train['Q13'] + train['Q14'] +train['Q17'] + train['Q18'] + train['Q20']
train['M'] = train['Q9'] + train['Q19']

###
q_ls = [[1,2,3],[1,2,3,4],[1,2,3],[1,2,3],[1,2,3],[1,2,3],[1,2,3,4],[1,2,3],[1,2,3],[1,2,3],[1,2],[1,2,3],[1,2],[1,2,3],[1,2],[1,2],[1,2,3],[1,2,3],[1,2,3],[1,2],[1,2,3],[1,2],[1,2,3],[1,2,3,4],[1],[1,2,3,4]]
for i in range(1,27):
    if i == 1:
        train['Q_normal'] = train['Q'+str(i)].apply(lambda x : 1 if x in q_ls[i-1] else 0)
    else:
        train['Q_normal'] += train['Q'+str(i)].apply(lambda x : 1 if x in q_ls[i-1] else 0)


train['introelapse'] = np.log1p(train['introelapse'])
train['testelapse'] = np.log1p(train['testelapse'])
train['surveyelapse'] = np.log1p(train['surveyelapse'])

###
train['age_over_50'] = train['age'].apply(lambda x : 1 if x>=50 else 0)
train['age'] = np.log1p(train['age'])


train['nature_score'] = train[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : np.round(x.mean(),2),axis=1)
train_fill_na = train.fillna(train.mean())


#################################

test = pd.read_csv('./data/test.csv').drop('index',axis=1)

test.loc[test['familysize']>100,'familysize'] = train['familysize'].mean()
test.loc[test['age']>100,'age']= train['age'].mean()

value = test['country'].value_counts().values
rank = list(map(lambda x : 1 if x>1000 else (2 if x>100 else 3),value))

temp_dict = {i : 0 for i in test['country'].value_counts().index.to_list()}

rank_dict = dict(zip(test['country'].value_counts().index.to_list(), rank))
rank_dict['nan'] = 0
test['country'] = test['country'].fillna('nan')
test['country'] = test['country'].apply(lambda x : rank_dict[x])

test['Ex'] = (test['TIPI1']+test['TIPI6'])/2
test['Ag'] = (test['TIPI7']+test['TIPI2'])/2
test['Con'] = (test['TIPI3']+test['TIPI8'])/2
test['Es'] =(test['TIPI9']+test['TIPI4'])/2
test['Op'] =(test['TIPI5']+test['TIPI10'])/2

###
test['TIPI_normal'] = test['TIPI1'].apply(lambda x : 1 if x==4 else 0)
test['TIPI_normal'] += test['TIPI2'].apply(lambda x : 1 if x==2 else 0)
test['TIPI_normal'] += test['TIPI10'].apply(lambda x : 1 if (x==3 or x== 4) else 0)
test['Es_gender'] = test['Es']*test['gender']
test['Con_gender'] = test['Con']*test['gender']
test['Op_gender'] = test['Op']*test['gender']
# test['religion'] = test['religion'].apply(lambda x : 1 if (x==1 or x==2) else 2)
test['religion'] = np.log1p(test['religion'])

test['mach_score'] = test[test.columns[:20]].apply(lambda x : x.mean(),axis=1)
test['T'] = test['Q1'] + test['Q2'] + test['Q3'] + test['Q6'] + test['Q7'] + test['Q10'] + test['Q12'] + test['Q15'] + test['Q16']
test['V'] = test['Q4'] + test['Q5'] + test['Q8'] + test['Q11'] + test['Q13'] + test['Q14'] + test['Q17'] + test['Q18'] + test['Q20']
test['M'] = test['Q9'] + test['Q19']

###
for i in range(1,27):
    if i == 1:
        test['Q_normal'] = test['Q'+str(i)].apply(lambda x : 1 if x in q_ls[i-1] else 0)
    else:
        test['Q_normal'] += test['Q'+str(i)].apply(lambda x : 1 if x in q_ls[i-1] else 0)

test['introelapse'] = np.log1p(test['introelapse'])
test['testelapse'] = np.log1p(test['testelapse'])
test['surveyelapse'] = np.log1p(test['surveyelapse'])

###
test['age_over_50'] = test['age'].apply(lambda x : 1 if x>=50 else 0)
test['age'] = np.log1p(test['age'])

test['nature_score'] = test[[('Q'+str(i)) for i in range(20,27)]].apply(lambda x : np.round(x.mean(),2),axis=1)

test_fill_na = test.fillna(train.mean())

In [3]:
temp_ls = train_fill_na.columns.to_list()
temp_ls.remove('mach_score')
temp_ls.remove('nature_score')
temp_ls.remove('nerdiness')
temp_ls.remove('country')
temp_ls.remove('introelapse')
temp_ls.remove('testelapse')
temp_ls.remove('surveyelapse')
temp_ls.remove('hand')

In [4]:
for i in temp_ls:
    train_fill_na[i] = train_fill_na[i].astype(int)
    test_fill_na[i] = test_fill_na[i].astype(int)

In [5]:
y_train = train_fill_na['nerdiness']
x_train = train_fill_na.drop('nerdiness',axis=1)

test = test_fill_na

In [19]:
x_train['nerdiness'] = train_fill_na['nerdiness']

In [1]:
!pip install xlearn

  Using cached xlearn-0.40a1.tar.gz (4.9 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      /home/kimjinsung/anaconda3/envs/for_pytorch/lib/python3.9/site-packages/setuptools/dist.py:516: UserWarning: Normalizing '0.40.a1' to '0.40a1'
        warnings.warn(tmpl.format(**locals()))
      running bdist_wheel
      running build
      running build_py
      Traceback (most recent call last):
        File "/tmp/pip-install-fxgnwa2m/xlearn_fb8804f019784afe99753b5bca704e17/setup.py", line 37, in silent_call
          subprocess.check_output(cmd, stderr=shut_up)
        File "/home/kimjinsung/anaconda3/envs/for_pytorch/lib/python3.9/subprocess.py", line 424, in check_output
          return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
        File "/home/kimjinsung/anaconda3/envs/for_pytorch/lib/python3.9/subprocess.py", line 505, in run
 

In [7]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# from sklearn.ensemble import ExtraTreesClassifier
from rgf.sklearn import RGFClassifier

import warnings
import gc
warnings.filterwarnings("ignore")

In [16]:
def lgbm_rfe_119(x_data, y_data, ratio=0.975, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = RGFClassifier(max_leaf=300,n_jobs=64)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=77, stratify=y_data)
        model.fit(x_train, y_train)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [17]:
lgbm_archive_4040 = lgbm_rfe_119(x_train, y_train)

84 0.7933762108611554
81 0.7933762108611554
78 0.7933762108611554
76 0.7933762108611554
74 0.7933762108611554
72 0.7933762108611554
70 0.7933762108611554
68 0.7933762108611554
66 0.7933762108611554
64 0.7933762108611554
62 0.7933762108611554
60 0.7933762108611554
58 0.7938774852093835
56 0.7936808336414112
54 0.7926566907024981
52 0.7941083996420539
50 0.7937513748334364
48 0.7916008762079819
46 0.7931556616648643
44 0.7941587862077862
42 0.7932224598548637
40 0.793253843487234


In [42]:
model = ExtraTreesClassifier(n_estimators=1000, n_jobs=64, random_state=119)

x_train_1 = x_train[lgbm_archive_4040.iloc[lgbm_archive_4040[lgbm_archive_4040['score']==lgbm_archive_4040['score'].max()].index[0],2]]

model.fit(x_train_1, y_train)

pred_y1 = model.predict_proba(test[lgbm_archive_4040.iloc[lgbm_archive_4040[lgbm_archive_4040['score']==lgbm_archive_4040['score'].max()].index[0],2]])
pred_y1 = pred_y1[:,1]

In [52]:
pred_all = pred_y1

index = pd.read_csv('./data/test.csv').index.to_list()

submission = pd.DataFrame({
    "index" : index,
    "nerdiness" : pred_all
})
submission.to_csv('./data/model2_extra_tree_add_features.csv', index=False)